In [1]:
!python -m venv venv

In [4]:
!venv\Scripts\activate

In [ ]:
%pip install pyodbc pandas
%pip install adodbapi

In [11]:
import adodbapi
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
# import matplotlib.pyplot as plt

In [7]:
server = "REVISION-PC"
catalog = "2023_CQ_BI_13_OLAP"
cube = "CQ BI 13 DDS"

conn_str = f"Provider=MSOLAP;Data Source={server};Initial Catalog={catalog};"

In [8]:
# Predict the number of products that will be sold
try:
    with adodbapi.connect(conn_str) as conn:
        cursor = conn.cursor()
        
        sql_stmt = """
            SELECT NON EMPTY { [Measures].[Total Sold Products] } ON COLUMNS, 
            NON EMPTY { 
                (
                    [Dim Product].[Product Id].[Product Id].ALLMEMBERS
                    * [Dim Branch].[Branch Id].[Branch Id].ALLMEMBERS
                    * [Dim Payment Type].[Payment Type Id].[Payment Type Id].ALLMEMBERS 
                    * [Dim Customer Type].[Customer Type Id].[Customer Type Id].ALLMEMBERS
                    * [Dim Datetime].[Day Number].[Day Number].ALLMEMBERS 
                    * [Dim Datetime].[Month Number].[Month Number].ALLMEMBERS 
                    * [Dim Datetime].[Year Number].[Year Number].ALLMEMBERS 
                ) 
            } DIMENSION PROPERTIES MEMBER_CAPTION, MEMBER_UNIQUE_NAME ON ROWS 
            FROM [CQ BI 13 DDS]
        """

        cursor.execute(sql_stmt)
        rows = cursor.fetchall()

    result = [
        {
            "product_id": row[0],
            "branch_id": row[2],
            "payment_type_id": row[4],
            "customer_type_id": row[6],
            "day": row[8],
            "month": row[10],
            "year": row[12],
            "total_sold_products": row[14]
        } for row in rows
    ]

    df = pd.DataFrame(result)
    y = df["total_sold_products"]
    X = df.drop(["total_sold_products"], axis=1)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    model = LinearRegression()
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    mse = mean_squared_error(y_test, predictions)
    print(f"Mean Squared Error: {mse}")

except adodbapi.DatabaseError as ex:
    print("Connection error:", ex)


Mean Squared Error: 9.921166522352237


In [9]:
prediction = model.predict([[2, 1, 3, 1, 27, 1, 2019]])
print(prediction)

[5.55171889]


c:\Users\Thuan\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


In [29]:
# # Detect shopping trends and predict sales.

# with adodbapi.connect(conn_str) as conn:
#     cursor = conn.cursor()
    
#     sql_stmt = """
#         SELECT NON EMPTY { 
#             [Measures].[Total Sold Products], 
#             [Measures].[Total Rating Score], 
#             [Measures].[Total Purchase Times], 
#             [Measures].[Total Income], 
#             [Measures].[Total Female Customers] 
#         } ON COLUMNS, 
#         NON EMPTY { 
#             (
#                 [Dim Product].[Product Id].[Product Id].ALLMEMBERS
#             )
#         } DIMENSION PROPERTIES MEMBER_CAPTION, MEMBER_UNIQUE_NAME ON ROWS 
#         FROM [CQ BI 13 DDS]
#     """

#     cursor.execute(sql_stmt)
#     rows = cursor.fetchall()

# result = []

# for row in rows:
#     result.append(
#         {
#             "product_id": row[0],
#             "total_sold_products": row[2],
#             "total_rating_score": row[3],
#             "total_purchase_times": row[4],
#             "total_income": row[5],
#             "total_female_customers": row[6]
#         }
#     )

# selected_columns = [
#     'total_sold_products', 
#     'total_purchase_times', 
#     'total_income', 
#     'total_female_customers', 
#     'product_id',
#     'total_rating_score'
# ]
# df = pd.DataFrame(result)
# df = df.dropna(subset=selected_columns)

# X = df[selected_columns[:-1]]
# y = df['total_rating_score']

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# model = LogisticRegression(random_state=42, max_iter=1000)
# model.fit(X_train, y_train)

# y_pred = model.predict(X_test)

# accuracy = accuracy_score(y_test, y_pred)
# report = classification_report(y_test, y_pred)

# print(f"Accuracy: {accuracy}")
# print("Classification Report:")
# print(report)


ValueError: continuous is not supported